In [1]:
import torch as T
import torch.nn as nn
from torch import optim
import torchvision.models as models
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, ConcatDataset, IterableDataset
import numpy as np
import pandas as pd
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [2]:
def get_cuda(tensor):
    if T.cuda.is_available():
        tensor = tensor.cuda()
    return tensor

In [3]:
class Encoder(nn.Module):
    ''' C-RNN-GAN generator
    '''
    def __init__(self, num_feats, hidden_units, drop_prob=0.6, use_cuda=True):
        super(Encoder, self).__init__()

        # params
        self.hidden_dim = hidden_units
        self.use_cuda = use_cuda
        self.num_feats = num_feats

        #self.fc_layer1 = nn.Linear(in_features=num_feats, out_features=hidden_units)
        self.lstm1 = nn.LSTM(num_feats, 600, batch_first=True)
        self.lstm2 = nn.LSTM(600, 500,2, batch_first=True)
        #self.dropout = nn.Dropout(p=drop_prob, batch_first=True)
        self.lstm3 = nn.LSTM(500, 400,2, batch_first=True)
        

    def forward(self, z):
        ''' Forward prop
        '''
        if self.use_cuda:
            z = z.cuda()

        h, (h_n, c_n) = self.lstm1(z)
        h, (h_n, c_n) = self.lstm2(h)
        h, (h_n, c_n) = self.lstm3(h)
        return h, (h_n, c_n)

In [4]:
class Generator(nn.Module):
    ''' C-RNN-GAN generator
    '''
    def __init__(self, num_feats, hidden_units, drop_prob=0.6, use_cuda=True):
        super(Generator, self).__init__()

        # params
        self.hidden_dim = hidden_units
        self.use_cuda = use_cuda
        self.num_feats = num_feats

        self.lstm1 = nn.LSTM(400, 500, batch_first=True)
        self.lstm2 = nn.LSTM(500, 1,2, batch_first=True)
        self.fc_layer1 = nn.Linear(in_features=1, out_features=1)
        #self.fc_layer2 = nn.Linear(in_features=1, out_features=1)

    def forward(self, z,states):
        ''' Forward prop
        '''
        (h0,c0)=states
        if self.use_cuda:
            z = z.cuda()
            h0=h0.cuda()
            c0=c0.cuda()
        batch_size, seq_len, num_feats = z.shape
        h, (h_n, c_n) = self.lstm1(z)
        h, (h_n, c_n) = self.lstm2(h)
        fc_output=self.fc_layer1(h)
        #fc_output=self.fc_layer2(fc_output)
        return fc_output

In [5]:
class Discriminator(nn.Module):
    ''' C-RNN-GAN discrminator
    '''
    def __init__(self, num_feats, hidden_units, use_cuda=False):

        super(Discriminator, self).__init__()

        # params
        self.hidden_dim = hidden_units
        self.num_layers = 1
        self.use_cuda = use_cuda

        self.lstm = nn.LSTM(input_size=1, hidden_size=10,
                            num_layers=self.num_layers, batch_first=True,bidirectional=True)
        self.fc_layer1 = nn.Linear(in_features=(2*10), out_features=10)
        self.fc_layer2 = nn.Linear(in_features=10, out_features=5)
        self.fc_layer3 = nn.Linear(in_features=5, out_features=1)

    def forward(self, note_seq,flag):
        ''' Forward prop
        '''
        #print("FORWARD")
        if self.use_cuda:
            note_seq = note_seq.cuda()

        lstm_out, state = self.lstm(note_seq)
        (h_n,c_n)=state
        h_n = T.cat((h_n[0],h_n[1]),dim=1)
        out = self.fc_layer1(h_n)
        out = self.fc_layer2(out)
        out = self.fc_layer3(out)
        num_dims = len(out.shape)
        reduction_dims = tuple(range(1, num_dims))
        out = T.mean(out, dim=reduction_dims)
        
        return out

In [6]:
# Create Dataset
class CSVDataset(Dataset):
    def __init__(self, path, chunksize,length,seq_len):
        self.path = path
        self.chunksize = chunksize
        self.len = int(length)#number of times total getitem is called
        self.seq_len=seq_len
        self.reader=pd.read_csv(
                self.path,header=0,
                chunksize=self.chunksize)#,names=['data']))

    def __getitem__(self, index):
        data = self.reader.get_chunk(self.chunksize)
        #sex=pd.read_csv('C:\\Users/mehak/Desktop/demo.csv',header=0)
        #sex=sex[['person_id','Sex']]
        #data = pd.merge(data, sex, how='left', on=['person_id'])
        del data['person_id']
           
        data=data.replace(np.nan,0)
        
        data = T.as_tensor(data.values.astype(float), dtype=T.float32)
        data=data.view(int(data.shape[0]/self.seq_len), self.seq_len, data.shape[1])
        return data

    def __len__(self):
        return self.len

In [7]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model, optimizer, save_path):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model, optimizer, save_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model, optimizer, save_path)
            self.counter = 0

    def save_checkpoint(self, val_loss, model, optimizer, save_path):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        T.save({
            "E_model": model['e'].state_dict(),
            "G_model": model['g'].state_dict(),
            "D_model": model['d'].state_dict(),
            'E_trainer': optimizer['e'].state_dict(),
            'G_trainer': optimizer['g'].state_dict(),
            'D_trainer': optimizer['d'].state_dict()
        }, save_path)
        self.val_loss_min = val_loss